# 10 · EDA Baseline
Margins & team trends.

In [ ]:
import os, glob
import pandas as pd
import plotly.express as px

# Load data (assumes repo root)
files = sorted(glob.glob("data/exports/**/*.parquet", recursive=True))
if not files:
    print("No parquet found. If running in Colab, run 00_colab_bootstrap first with USE_DB=True or generate exports.")
    df = pd.DataFrame()
else:
    df = pd.concat((pd.read_parquet(p) for p in files), ignore_index=True)

assert not df.empty, "No data loaded."

df["home_margin"] = df["home_score"] - df["away_score"]
long = pd.concat([
    df[["season","home_team","home_margin"]].rename(columns={"home_team":"team","home_margin":"margin"}),
    df[["season","away_team","home_margin"]].assign(home_margin=lambda x: -x["home_margin"]).rename(columns={"away_team":"team","home_margin":"margin"})
], ignore_index=True)

by_team = long.groupby(["season","team"], as_index=False)["margin"].mean()

fig = px.line(by_team, x="season", y="margin", color="team", title="Average margin by team (per season)")
fig.show()


In [ ]:
# Export HTML report
!jupyter nbconvert --to html --output-dir=reports --no-input notebooks/10_eda_baseline.ipynb
